<a href="https://colab.research.google.com/github/Lucia1009/TFG-Plataforma-para-la-prediccion-de-potencia-electrica-generada-en-parques-eolicos/blob/desarrollo/modelos/YDF_datos_variados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install dash -q
!pip install ydf -q

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import dash
from dash import Dash, html, dcc, Input, Output
import plotly.express as px
import plotly.graph_objects as go

import ydf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.6/10.6 MB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.7/319.7 kB 11.9 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df1=pd.read_csv('/content/drive/MyDrive/TFG/datos_variaciones/Report_ElValle_cada_minuto_sinteticos_con_esperas_transitorio_aa.csv', delimiter=";",decimal=".",skiprows = [1])
df2=pd.read_csv('/content/drive/MyDrive/TFG/datos_variaciones/Report_ElValle_cada_minuto_sinteticos_con_esperas_transitorio_ab.csv', delimiter=";",decimal=".",skiprows = [1])
df3=pd.read_csv('/content/drive/MyDrive/TFG/datos_variaciones/Report_ElValle_cada_minuto_sinteticos_con_esperas_transitorio_ac.csv', delimiter=";",decimal=".",skiprows = [1])
df4=pd.read_csv('/content/drive/MyDrive/TFG/datos_variaciones/Report_ElValle_cada_minuto_sinteticos_con_esperas_transitorio_ad.csv', delimiter=";",decimal=".",skiprows = [1])
df5=pd.read_csv('/content/drive/MyDrive/TFG/datos_variaciones/Report_ElValle_cada_minuto_sinteticos_con_esperas_transitorio_ae.csv', delimiter=";",decimal=".",skiprows = [1])

df=pd.concat([df1,df2,df3,df4,df5])

# Tratamiento de datos

In [4]:
df.head()
print(df.shape)

# Filtro para el régimen estacionario
datos_filtrado=df[(df['TIME'] % 600==540)]

print(datos_filtrado.shape)

#Reseteo el índice y quito la primera fila
datos_filtrado = datos_filtrado.reset_index(drop=True)
print(datos_filtrado.iloc[0]['TIME'])
print(datos_filtrado.iloc[1]['TIME'])

datos_filtrado=datos_filtrado.drop(0)
print(datos_filtrado.iloc[0]['TIME'], '\n')

#Obtener la dirección wd mímina y máxima de datos_filtrado
min=datos_filtrado['wd'].min()
max=datos_filtrado['wd'].max()
print(min, max)

datos_filtrado = datos_filtrado[datos_filtrado['wd'] >= 0]
datos_filtrado = datos_filtrado[datos_filtrado['wd'] < 360]
print(datos_filtrado.shape)
min=datos_filtrado['wd'].min()
max=datos_filtrado['wd'].max()
print(min, max, '\n')

# Obtener el mínimo y máximo de potencia
min=datos_filtrado['WF_Power'].min()
max=datos_filtrado['WF_Power'].max()
print(min, max)

datos_filtrado = datos_filtrado[datos_filtrado['WF_Power'] >= 0]
print(datos_filtrado.shape, '\n')

#Obtener el mínimo y máximo de velocidad de viento ws
min=datos_filtrado['ws'].min()
max=datos_filtrado['ws'].max()
print(min, max)

datos_filtrado = datos_filtrado[datos_filtrado['ws'] >= 0]
print(datos_filtrado.shape, '\n')

datos_filtrado.head()

(1000000, 165)
(100000, 165)
540.0
1140.0
1140.0 

-15.0213314 372.700618
(99363, 165)
0.0137177188 359.995667 

-0.000116567547 70053460.3
(92619, 165) 

-3.02420751e-07 30.0000369
(92600, 165) 



,TIME,TC1.T.Pg.signal[1],TC2.T.Pg.signal[1],TC3.T.Pg.signal[1],TC4.T.Pg.signal[1],TC5.T.Pg.signal[1],TC6.T.Pg.signal[1],TC7.T.Pg.signal[1],TC8.T.Pg.signal[1],TC9.T.Pg.signal[1],...,TC5.set_point.signal[1],TC6.set_point.signal[1],TC7.set_point.signal[1],TC8.set_point.signal[1],TC9.set_point.signal[1],TC10.set_point.signal[1],TC11.set_point.signal[1],TC12.set_point.signal[1],TC13.set_point.signal[1],TC14.set_point.signal[1]
1,1140,1256141.37,1256258.27,1254315.43,1254475.38,1256258.27,1119599.45,1254259.68,1256258.27,1256258.27,...,1266053.06,1143658.70,1266053.06,1266053.06,1266053.06,1161065.93,1132624.40,1109987.84,992630.454,995151.314
2,1740,1258355.96,1258355.97,1258360.59,1102833.33,1258355.97,1237839.61,1258478.81,1258355.97,1258355.97,...,1258549.76,1237829.90,1258549.76,1258549.76,1258549.76,1186955.37,1084856.81,1048207.32,1057708.930,1135283.740
3,2340,1190767.16,1190767.16,1190767.16,1190979.42,1190767.16,1190768.98,1190767.18,1190732.19,1190767.16,...,1190253.08,1190253.08,1190253.08,1190253.08,1190253.08,1190253.08,1190253.08,1190253.08,1190253.080,1190253.080
4,2940,1212111.16,1212111.16,1212111.16,1212138.75,1212111.16,1212111.16,1212111.16,1121821.05,1212111.16,...,1212733.26,1212733.26,1212733.26,1122172.65,1212733.26,1129804.87,1097180.22,1187052.81,1212733.260,1212733.260
5,3540,1191981.01,1191981.01,1191981.01,1191981.01,1191981.01,1191981.01,1191981.01,1191535.76,1191586.39,...,1190253.08,1190253.08,1190253.08,1190253.08,1190253.08,1190253.08,1190253.08,1190253.08,1190253.080,1190253.080


In [5]:
# Obtengo los distintos valores para cada setpoint

setpoints=datos_filtrado['WF_Power_SP'].unique()
print(setpoints)
print(len(setpoints))
print('type: ', type(setpoints))


[16663543.2 42690226.6 14844327.7 ... 41590025.8 72250076.  38639524.4]
2820
type:  <class 'numpy.ndarray'>


In [6]:
datos=datos_filtrado[['ws', 'wd', 'WF_Power', 'WF_Power_SP']]
datos['sin_wd'] = np.sin(datos['wd']*np.pi/180)
datos['cos_wd'] = np.cos(datos['wd']*np.pi/180)
datos = datos.drop('wd', axis=1)

#### DATOS ESTRATIFICADOS ####

datos_train_rf = []
datos_test_rf = []
for i in range(datos.shape[0]):

  if i%4==0:
      datos_test_rf.append(datos.iloc[i])
  else:
      datos_train_rf.append(datos.iloc[i])

print(len(datos_train_rf), len(datos_test_rf))
print(datos.shape[0]/len(datos_test_rf))
print(datos.shape[0]/4)

print(datos_train_rf[0], "\n", datos_train_rf[1])

dftrain = pd.DataFrame(datos_train_rf)
dftest = pd.DataFrame(datos_test_rf)


<ipython-input-6-cf60fcecca54>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datos['sin_wd'] = np.sin(datos['wd']*np.pi/180)
<ipython-input-6-cf60fcecca54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datos['cos_wd'] = np.cos(datos['wd']*np.pi/180)


69450 23150
4.0
23150.0
ws             1.104202e+01
WF_Power       1.666233e+07
WF_Power_SP    1.666354e+07
sin_wd         5.281931e-01
cos_wd        -8.491243e-01
Name: 2, dtype: float64 
 ws             1.223990e+01
WF_Power       1.667089e+07
WF_Power_SP    1.666354e+07
sin_wd         4.231646e-01
cos_wd        -9.060528e-01
Name: 3, dtype: float64


In [7]:
def obtener_direcciones(df_filtrado, columnas):

  direcciones={}
  for i in range(0,360):
      # Separo los datos para cada dirección
      filtered_data = df_filtrado[(df_filtrado['wd'] <= i) & (df_filtrado['wd'] > (i - 1 ))]

      # Selecciono solo las columnas que me interesan
      selected_columns = filtered_data[columnas]

      direcciones[i]=selected_columns

  return direcciones


In [8]:
#### MODELO ####

learner = ydf.RandomForestLearner(label="WF_Power", task = ydf.Task.REGRESSION,
                                    num_trees=301, max_depth=10,
                                    split_axis="SPARSE_OBLIQUE",categorical_algorithm="RANDOM",
                                    missing_value_policy='RANDOM_LOCAL_IMPUTATION',
                                    sparse_oblique_normalization='STANDARD_DEVIATION',
                                    compute_oob_variable_importances=True,
                                    winner_take_all=False)
model_le=learner.train(dftrain)
# evaluation = learner.cross_validation(dftest)

# # Print the evaluation
# print(evaluation)


# # model_le.evaluate(dftest)  #Equivale a imprimir la evaluación

Train model on 69450 examples
Model trained in 0:03:21.521649


In [9]:
# evaluation = learner.cross_validation(dftest)

evaluation=model_le.evaluate(dftest)  #Equivale a imprimir la evaluación

evaluation


Evaluation()

In [10]:
# Get the predictions
predictions = model_le.predict(dftest)

# Access the actual target values from the 'dftest' DataFrame
labels = dftest["WF_Power"].values  # Replace "WF_Power" with the actual target column name if different

# Calculate the residuals
residuals = predictions.flatten() - labels

# Create the interactive histogram
fig = px.histogram(
    x=residuals,
    nbins=800,  # Adjust the number of bins as needed
    title="Histograma Interactivo de Residuos",
    labels={"x": "Residuos", "y": "Frecuencia"},
)


# Calcula el percentil 5 y 95
percentil_5 = np.percentile(residuals, 5)
percentil_95 = np.percentile(residuals, 95)

# Agrega las líneas verticales
fig.add_vline(x=percentil_5, line_width=3, line_dash="dash", name="Percentil 5", line_color="red")
fig.add_vline(x=percentil_95, line_width=3, line_dash="dash", line_color="red")

# Calcula el percentil 5 y 95
percentil_10 = np.percentile(residuals, 10)
percentil_90 = np.percentile(residuals, 90)

# Agrega las líneas verticales
fig.add_vline(x=percentil_10, line_width=3, line_dash="dash", line_color="green")
fig.add_vline(x=percentil_90, line_width=3, line_dash="dash", line_color="green")



# Show the interactive histogram
fig.show()
# El rango de valores en el que cae el 90% de las muestras es:
print(f"Rango del 90% de las muestras: [{percentil_5}, {percentil_95}]")

# Guardar el histograma en un aRCHIVO
fig.write_html("histograma_interactivo.html")

Rango del 90% de las muestras: [-2008338.8799999966, 1315940.1850000005]


# Gráfica datos mantenidos

In [34]:
#### DATOS MANTENIDOS TEST ####

datos_test_rf = []
datos_test_rf_label = []
for i in range(datos.shape[0]):

  if i%4==0:
      datos_test_rf_label.append(datos.iloc[i-1]['WF_Power'])
      datos_test_rf.append(datos.iloc[i])


dftest_mantenida = pd.DataFrame(datos_test_rf)
dftest_mantenida['WF_Power']=datos_test_rf_label

evaluacion_mantenida=model_le.evaluate(dftest_mantenida)

evaluacion_mantenida

Evaluation()

In [50]:
# Get the predictions and residuals for both datasets
predictions = model_le.predict(dftest)
labels = dftest["WF_Power"].values
residuals = predictions.flatten() - labels

predictions_mantenida = model_le.predict(dftest_mantenida)
labels_mantenida = dftest_mantenida["WF_Power"].values
residuals_mantenida = predictions_mantenida.flatten() - labels_mantenida

# Create the histogram traces

trace1 = go.Histogram(
    x=residuals,
    nbinsx=800,
    name="Datos normales",
    marker=dict(color="blue"),
    opacity=1,
)

trace2 = go.Histogram(
    x=residuals_mantenida,
    nbinsx=800,
    name="Datos mantenidos",
    marker=dict(color="red"),
    opacity=0.7,
)

# Calculate percentiles for both datasets
percentiles = [5, 10, 90, 95]
percentiles_normales = np.percentile(residuals, percentiles)
percentiles_mantenidos = np.percentile(residuals_mantenida, percentiles)

# Create the layout
layout = go.Layout(
    title="Histograma de Residuos (Datos Normales vs. Mantenidos)",
    xaxis_title="Residuos",
    yaxis_title="Frecuencia",
    barmode="overlay",  # Overlay histograms
    # legend=dict(x=0.7, y=0.9),  # Adjust legend position if necessary

)

# Create the figure and add traces
fig = go.Figure(data=[trace1, trace2], layout=layout)

# Add vertical lines using fig.add_vline() with name for legend
for i, p in enumerate(percentiles):
    # Para datos normales
    fig.add_vline(
        x=percentiles_normales[i],
        line_width=3,
        line_dash="dash",
        line_color="blue",
        name=f"Percentil {p} (normales)",  # Nombre para la leyenda
    )
    # Para datos mantenidos
    fig.add_vline(
        x=percentiles_mantenidos[i],
        line_width=3,
        line_dash="dash",
        line_color="red",
        name=f"Percentil {p} (mantenidos)",  # Nombre para la leyenda
    )

fig.update_layout(legend=dict(x=0.7, y=0.9))


# Create annotations for the text output
annotations = [
    dict(
        x=0.01,  # Adjust x-coordinate as needed
        y=0.95,  # Adjust y-coordinate as needed
        xref="paper",  # Position relative to the figure
        yref="paper",  # Position relative to the figure
        text=f"Rango del 90% mantenidos: [{percentiles_mantenidos[0]}, {percentiles_mantenidos[1]}]<br>"
             f"Rango del 90% normales: [{percentiles_normales[0]}, {percentiles_normales[1]}]<br><br>"
             f"Rango del 80% mantenidos: [{percentiles_mantenidos[2]}, {percentiles_mantenidos[3]}]<br>"
             f"Rango del 80% normales: [{percentiles_normales[2]}, {percentiles_normales[3]}]<br><br>"
             f"El RMSE mantenido es: {evaluacion_mantenida.rmse}<br>"
             f"El RMSE normal es    : {evaluation.rmse}",
        showarrow=False,  # Hide arrow
        font=dict(size=12),  # Adjust font size as needed
    )
]

# Update layout with annotations
fig.update_layout(annotations=annotations)

# Show and save the figure
fig.show()
fig.write_html("histograma_unificado.html")

In [14]:
#### GRÁFICA RANDOM FOREST YDF ####

app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Gráfica de Potencia vs. Velocidad"),
    dcc.Input(
        id='angulo',
        type='number',
        value=0,
        placeholder='Introduce el ángulo'
    ),
    dcc.Graph(id='grafico-potencia-velocidad')
])

@app.callback(
    Output('grafico-potencia-velocidad', 'figure'),
    [Input('angulo', 'value')]
)
def update_graph(angulo):

    sin_angulo_val = np.sin(angulo * np.pi / 180)
    cos_angulo_val = np.cos(angulo * np.pi / 180)

    # Filtrar todos los datos con base en el ángulo
    datos_filtrados_total = datos[(datos['sin_wd'] == sin_angulo_val) &
                              (datos['cos_wd'] == cos_angulo_val)]

    # Comprobar si el filtrado contiene datos
    if not datos_filtrados_total.empty:
        # Ordenar por velocidad del viento (ws)
        datos_filtrados_total = datos_filtrados_total.sort_values(by='ws')
        y_filtrados_total = datos_filtrados_total['WF_Power']

        # Filtrar los datos de test
        datos_filtrados_graf = dftest[(dftest['sin_wd']== sin_angulo_val) &
                                      (dftest['cos_wd']== cos_angulo_val)]


        y_filtrado_graf = datos_filtrados_graf['WF_Power']


        # Hacer predicciones con el modelo
        y_pred_filtrado_graf = model_le.predict(datos_filtrados_graf)


        # Crear la gráfica con los datos reales y las predicciones
        fig = go.Figure()

        # Datos reales (puntos)
        fig.add_trace(go.Scatter(x=datos_filtrados_graf['ws'], y=y_filtrado_graf,
                                  mode='markers', name='Potencia real',
                                  marker=dict(color='blue')))

        # Predicciones (puntos)
        fig.add_trace(go.Scatter(x=datos_filtrados_graf['ws'], y=y_pred_filtrado_graf.flatten(),
                                  mode='markers', name='Potencia predicha',
                                  marker=dict(color='red')))

        rango_x_graf = datos_filtrados_total['ws']

        # Predecir los valores en el rango
        rango_y_pred_graf = model_le.predict(datos_filtrados_total[['ws', 'sin_wd', 'cos_wd']])
        # Interpolación de los valores reales
        rango_y_real_graf = np.interp(rango_x_graf, datos_filtrados_total['ws'], y_filtrados_total)

        # Añadir las líneas a la gráfica
        fig.add_trace(go.Scatter(x=rango_x_graf, y=rango_y_pred_graf, mode='lines', name='Línea predicción', line=dict(color='red')))
        fig.add_trace(go.Scatter(x=rango_x_graf, y=rango_y_real_graf, mode='lines', name='Línea real', line=dict(color='blue')))


        fig.update_layout(
            title="Potencia vs. Velocidad",
            xaxis_title="Velocidad del viento (ws)",
            yaxis_title="Potencia (WF_Power)"
        )

        return fig
    else:
        # Si no hay datos filtrados, devuelve una gráfica vacía
        return px.scatter(x=[], y=[],
                          labels={'x': 'Velocidad del viento (ws)', 'y': 'Potencia (WF_Power)'})

if __name__ == '__main__':
    app.run_server(debug=True, port=8052)



<IPython.core.display.Javascript object>